# Diet Problem
Based on chapter 2 of the AMPL book and section 6.2 of *Decision Making, Models and Algorithms: A First Course* by Saul I. Gass.

**Objectives**
- Introduce a historically important linear program
- Think about solving linear programs and tweaking solutions
- Demonstrate the idea of sensitivity analysis in linear programming

**Reading:** This lab uses a diet problem instance provided by NEOS. Their online diet problem solver can be found [HERE](https://neos-guide.org/content/diet-problem-solver). Please read the following description of the diet problem and its history: https://neos-guide.org/content/diet-problem

**Brief description:** In this lab, we will consider one of the most famous (and one of the earliest) applications of linear programming — the diet problem.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

In [ ]:
# imports -- don't forget to run this cell
import pandas as pd
from ortools.linear_solver import pywraplp as OR

## Part 1: The Diet Problem

Suppose that you have a choice of two cereals for breakfast: Krunchies (K) or Crispies (C). Breakfast is the
most important meal of the day, so you want to get sufficient Thiamine, Niacin, and caloric intake for breakfast. Being a college student, you want to do so as cheaply as possible – taste gets thrown into the wind! Price and nutritional info for these two cereals is summarized in the two tables below:

In [ ]:
cereals = pd.read_csv('data/diet_cereal/cereal_foods.csv', index_col=0)
cereal_nutrients = pd.read_csv('data/diet_cereal/cereal_nutrients.csv', index_col=0)
display(cereals)
display(cereal_nutrients)

**Q1:** Suppose you just ate Krunchies. How many ounces of Krunchies would you need to eat to satisfy the
three nutritional requirements? How much would this cost?

**A:**

**Q2:** Suppose you just ate Crispies. How many ounces of Crispies would you need to eat to satisfy the three
nutritional requirements? How much would this cost?

**A:** 

**Q3:** Now let’s write an optimization problem to model this problem. Let $K$ be a decision variable for the amount of Krunchies you eat and $C$ be a decision variable for the amount of Crispies you eat. Write an objective function, encoding that you minimize total cost (as a function of $K$ and $C$).

**A:** 

**Q4:** Write three constraints: one enforcing that you get at least 1 mg of Thiamine, one enforcing that you
get at least 5 mg of Niacin, and one enforcing that you get at least 400 calories. Also write constraints
that $K$ and $C$ are nonnegative.

**A:**

**Q5:** Implement this model in OR-Tools. The basic structure has been given to you.

In [ ]:
def small_diet():
    """A linear program for solving a diet problem."""
    # define the model
    m = OR.Solver('cereal_diet', OR.Solver.CLP_LINEAR_PROGRAMMING);

    # TODO: Define the decision variables.
    # name = m.NumVar(lower_bound, upper_bound, name)
    # If you want to use infinity as a bound, you can use m.infinity()


    # objective function
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()


    # constraints
    # TODO: Define the constraints.
    # m.Add()
    
    
    return m, m.variables()

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
m,x = small_diet()
solve(m)

**Q6** Run the cell above. What was the optimal solution and its objective value?

**A:** 

## Part 2: Generalizing the Diet Problem

We now know that if we expect to live a long and healthy life, we must have a well-balanced diet. Too much fat or sodium in our diet can lead to serious health problems. Similarly, diets deficient in essential vitamins and minerals should be avoided. 

This part of the lab is aimed at formulating this problem as a linear programming problem. We can view the diet problem in the following way. We are given a variety of foods that we could buy to achieve a balanced diet. For example, we might consider a diet consisting of 2% milk, spaghetti (with sauce), peanut butter, wheat bread, tomato soup, and bagels. To make things simpler, we will specify the variables of this linear programming formulation. We will use $x$_$\textit{(food-type)}$ to specify the number of daily servings of food-type that you are willing to consume. First of all, we wish to write constraints that capture whether one can satisfy certain daily requirements with just these foods.

**Q7:** Write constraints to ensure we eat at most 10 servings/day of each food-type (We can call this the boredom constraint).

**A:** 

Next, consider the total number of calories consumed. A proper diet requires that you consume between 2000 and 2250 calories per day. 

**Q8:** Write two constraints to ensure we eat an appropriate number of total calories. To write this constraint you need to know how many calories are in one serving of each of the food-types. Let $a_{ij}$ be the amount of nutrient $j$ in one serving of food $i$.

**A:**  

Of course, we could repeat this sort of constraint with any number of nutritional requirement, such as cholesterol, fat, sodium, dietary fiber, carbohydrates, protein, vitamin A, vitamin C, calcium, and iron.

**Q9:** The objective function is to minimize the cost of each day’s diet. Express this objective function in terms of the decision variables, given that $c_i$ is the cost of one serving of food $i$.

**A:** 

**Q10:** What does it mean if your linear program for this diet problem is infeasible?

**A:** 

**Q11:** How might you correct this?
    
**A:**

Let's use OR-Tools to implement this generalized diet problem model!

**Q12:** Complete the model below.

In [ ]:
def diet(foods, nutrients, integer=False):
    """A model for solving the diet problem.
    
    Args:
        foods (pd.DataFrame): Foods with cost per serving, min and max servings, and nutrients per serving.
        nutrients (pd.DataFrame): Nutrients with min and max bounds.
    """
    FOODS = list(foods.index)                                 # foods
    NUTRIENTS = list(nutrients.index)                         # nutrients
    c = foods['Cost'].to_dict()                               # cost per serving of food 
    f_min = foods['Min'].to_dict()                            # lower bound of food serving
    f_max = foods['Max'].to_dict()                            # upper bound of food serving
    n_min = nutrients['Min'].to_dict()                        # lower bound of nutrient
    n_max = nutrients['Max'].to_dict()                        # upper bound of nutrient  
    a = foods[list(nutrients.index)].transpose().to_dict()    # amt of nutrients per serving of food
    
    # define model
    m = OR.Solver('diet', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        
    # decision variables
    x = {}    
    for i in FOODS:
        if integer:
            x[i] = m.IntVar(0, m.infinity(), 'x_%s' % (i)) 
        else:
            x[i] = m.NumVar(0, m.infinity(), 'x_%s' % (i)) 
        
    # objective function.
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()

    
    # enforce lower and upper bound on food servings
    for i in FOODS:
        m.Add(x[i] >= f_min[i], name='lb_%s' % (i))
        m.Add(x[i] <= f_max[i], name='ub_%s' % (i))
    
    # enforce lower and upper bound on nutrients 
    for j in NUTRIENTS:
        m.Add(sum(a[i][j]*x[i] for i in FOODS) >= n_min[j], name='lb_%s' % (j))
        m.Add(sum(a[i][j]*x[i] for i in FOODS) <= n_max[j], name='ub_%s' % (j))
        
    return m, x

In [ ]:
def solve(m):
    '''Used to solve a model m.'''
    if m.Solve() == m.INFEASIBLE:
        print('infeasible.')
    else:
        print('Objective = %f \n' % (m.Objective().Value()))  
        x = {var.name() : var.solution_value() for var in m.variables()}
        for i in x:
            if x[i] != 0:
                print('%s: %s' % (i, x[i]))
                
def dual_values(m):
    '''Return the non-zero dual values.'''
    for i in m.constraints():
        name = i.name()
        dual_value = i.dual_value()
        if dual_value != 0:
            print(i.name(), i.dual_value())
            
def nutrient_values(x, m, foods, nutrients):
    '''Return the value of each nutrient in this solution.'''
    FOODS = list(foods.index)                                 
    NUTRIENTS = list(nutrients.index)                         
    a = foods[list(nutrients.index)].transpose().to_dict()    
    for j in NUTRIENTS:
        value = sum(a[i][j]*x[i].solution_value() for i in FOODS)
        print('%s: %f' % (j, value))

To make sure our model is correct, let's use it to solve the cereal input from **Part 1**. In order to use this new model, we will have to slightly modify our input. Recall the cereal input:

In [ ]:
display(cereals)
display(cereal_nutrients)

**Q13:** We need to specify an upper bound "Max" for each nutrient and both lower and upper bounds ("Min" and "Max") for each food. What shoud these be?

**A:** 

**Q14:** Implement your answer from **Q13**.

In [ ]:
# TODO: Set the values below.
# Hint: use float('inf') for infinity.

# cereals['Min'] =  
# cereals['Max'] = 
# nutrients['Max'] = 

In [ ]:
m,x = diet(cereals, cereal_nutrients)
solve(m)

**Q15:** Run the cell above to solve the model. Did you get the same solution as you did in **Part 1**?

**A:**

## Part 3: Solving the Diet Problem

Let's start by solving the diet problem input given in the beginning of **Part 2**. It uses a subset of all the foods in the NEOS dataset. Try to solve the model. You should see it is infeasible.

In [ ]:
foods = pd.read_csv('data/diet_neos/neos_foods.csv', index_col=0)
sm_foods = foods.loc[['2% Lowfat Milk', 'Spaghetti W/ Sauce', 
                      'Peanut Butter', 'Wheat Bread', 'Tomato Soup', 'Bagels']]
nutrients = pd.read_csv('data/diet_neos/neos_nutrients.csv', index_col=0)

In [ ]:
display(sm_foods.head())
display(nutrients)

In [ ]:
m,x = diet(sm_foods, nutrients)
solve(m)

**Q16:** Take a look at the specifics of the nutrient requirements and the nutrient contents, and try to understand why there is no feasible solution. What do think caused it to be infeasible? (I don’t know of a simple answer to this.)

**A:** 

In [ ]:
m,x = diet(foods, nutrients)
solve(m)

**Q17:** Now consider a much wider set of food-types (this diet problem is solved above). What is the optimal solution and objective value? (Hint: the solve function does not print foods with 0 servings in the solution.)

**A:**

**Q18:** How many different food-types are included in this diet (There were 62 potential foods)? Is this surprising to you?

**A:** 

In [ ]:
nutrient_values(x, m, foods, nutrients)

**Q19:** The function above returns how much of each nutrient you will actually consume. For which nutrients are you at your upper limit? Such a constraint is said to be tight.

**A:** 

**Q20:** For which are you at your lower limit? (That is, which lower limits are tight?)

**A:** 

**Q21:** Are any of the minimum and maximum number of serving constraints tight?

**A:**

You should have included carbohydrates in the list of nutrients that is at its upper limit in your diet. Use the function below to get the dual values of this solution. (Hint: this function only shows non-zero dual values.)

In [ ]:
dual_values(m)

**Q22:** Suppose you increase the allowance for carbohydrates to 301 grams. How does the optimal diet change? By how much does its cost change? How does this relate to the dual value for the constraint that upper bounds carbohydrates.

**A:** 

In [ ]:
tmp = nutrients.copy()
tmp.at['Carbohydrates', 'Max'] = 301
m,x = diet(foods, tmp)
solve(m)

Return to the dual information. The dual value for the carbohydrates upper bound constraints is given there. This information can be used in the following way. Suppose that a dual value for a particular nutrient's upper bound constraint is $−c$ , (i.e., it is negative). Now suppose that we may have one more unit of this nutrient in our diet, and when we find the optimal solution for this modified data, the foods that make up the optimal diet are unchanged.

**Q23:** How does the cost of the optimal solution to the new data compare to the optimal solution to the original data?    

**A:** 

**Q24:** In fact, the dual cost tells how much cheaper the new solution will be: it will be $c$ units cheaper. If we were to increase the max on total fat by 1 (making it 66), what would you expect the new cost to be? Solve the model to see if it was as expected.

**A:**  

In [ ]:
tmp = nutrients.copy()
tmp.at['Total_Fat', 'Max'] = 66
m,x = diet(foods, tmp)
solve(m)

**Q25:** Now allow up to 320 grams of carbohydrates in your diet. How does this change things? Does the analogous calculation to the one that you just did still work? Why do you think that this is?

**A:** 

In [ ]:
tmp = nutrients.copy()
tmp.at['Carbohydrates', 'Max'] = 320
m,x = diet(foods, tmp)
solve(m)

**Q26:** Go back to the default data, its optimal solution, and the dual and constraint information for it. Which constraint do you think it would be most profitable to violate by one unit? Can you give an intuitive explanation of this?
    
**A:**

**Q27:** As you can tell from the output, the optimal solution largely calls for non-integral numbers of servings of the various food-types. Why might this be problematic? Is there any explanation of the set-up for the problem that would allow us to consider such fractional solutions?

**A:** 

**Q28:** Suppose that you are buying just one day’s worth of food, and you must buy an integral number of servings of each food-type. Can you still use this output to construct a diet? Is this new selection an optimal diet subject to the restriction that the number of servings bought be integral?

**A:** 

**Q29:** Without worrying about how to compute the optimal integer solution, how does the cost of the optimal integer solution compare to the cost of our linear program’s optimal solution? 

**A:**

In [ ]:
m,x = diet(foods, nutrients, integer=True)
solve(m)

**Q30:** Solve the model with the restriction that servings must be integer. What is the optimal solution and objective value? How does it compare to the linear relaxation?

**A:** 

**Q31:** What do you think about eating the diet proposed by the output? How might you modify the constraints to get something more to your liking?

**A:**